In [ ]:
pip install transformers

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import urllib.request
from sklearn import preprocessing
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train=pd.read_csv('/content/drive/MyDrive/한국어문장분류공모전/DATA/train_final.csv')
test_data = pd.read_csv('/content/drive/MyDrive/한국어문장분류공모전/DATA/test_data.csv')
submission_data = pd.read_csv("/content/drive/MyDrive/한국어문장분류공모전/DATA/sample_submission.csv")

In [ ]:
train_data=train[:797848]
val_data=train[797848:]

In [ ]:
len(train_data)

797848

In [ ]:
train_data.head()

,premise,hypothesis,label
0,빨간 셔츠에 카키색 바지를 입은 남자가 그릴에서 저녁 식사를 고치고 있다.,빨간 셔츠와 카키스를 입은 남자가 밖에서 저녁을 만들고 있다.,entailment
1,호스트가 굉장히 관리를 잘하고 있는 곳입니다.,이곳의 관리가 굉장히 잘 되어있는 이유는 호스트 덕분입니다.,entailment
2,소녀 옆에 앉아 있는 어린 소년이 손으로 얼굴을 가리고 있다.,어린 소년이 어린 소녀에게 키스하려고 한다.,contradiction
3,"그래, 가끔은 좀 지루할 수도 있지만",나는 그 게임이 때때로 약간 지루할 수도 있다고 생각한다.,neutral
4,흰 토끼 두 마리가 초록색 풀 위에 앉아 있다.,두 마리의 토끼가 풀밭에서 논다.,neutral


훈련데이터와 테스트데이터의 샘플 개수 확인

In [ ]:
train_data.head()

,premise,hypothesis,label
0,빨간 셔츠에 카키색 바지를 입은 남자가 그릴에서 저녁 식사를 고치고 있다.,빨간 셔츠와 카키스를 입은 남자가 밖에서 저녁을 만들고 있다.,entailment
1,호스트가 굉장히 관리를 잘하고 있는 곳입니다.,이곳의 관리가 굉장히 잘 되어있는 이유는 호스트 덕분입니다.,entailment
2,소녀 옆에 앉아 있는 어린 소년이 손으로 얼굴을 가리고 있다.,어린 소년이 어린 소녀에게 키스하려고 한다.,contradiction
3,"그래, 가끔은 좀 지루할 수도 있지만",나는 그 게임이 때때로 약간 지루할 수도 있다고 생각한다.,neutral
4,흰 토끼 두 마리가 초록색 풀 위에 앉아 있다.,두 마리의 토끼가 풀밭에서 논다.,neutral


In [ ]:
train_data.head()

,premise,hypothesis,label
0,빨간 셔츠에 카키색 바지를 입은 남자가 그릴에서 저녁 식사를 고치고 있다.,빨간 셔츠와 카키스를 입은 남자가 밖에서 저녁을 만들고 있다.,entailment
1,호스트가 굉장히 관리를 잘하고 있는 곳입니다.,이곳의 관리가 굉장히 잘 되어있는 이유는 호스트 덕분입니다.,entailment
2,소녀 옆에 앉아 있는 어린 소년이 손으로 얼굴을 가리고 있다.,어린 소년이 어린 소녀에게 키스하려고 한다.,contradiction
3,"그래, 가끔은 좀 지루할 수도 있지만",나는 그 게임이 때때로 약간 지루할 수도 있다고 생각한다.,neutral
4,흰 토끼 두 마리가 초록색 풀 위에 앉아 있다.,두 마리의 토끼가 풀밭에서 논다.,neutral


In [ ]:
print('훈련용 샘플 개수 :',len(train_data))
print('검증용 샘플 개수 :',len(val_data))
print('테스트용 샘플 개수 :',len(test_data))

훈련용 샘플 개수 : 797848
검증용 샘플 개수 : 179462
테스트용 샘플 개수 : 1666


In [ ]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [ ]:
max_seq_len = 128

In [ ]:
prem = train_data['premise'].iloc[0]
hypo = train_data['hypothesis'].iloc[0]

print(prem)
print(hypo)

빨간 셔츠에 카키색 바지를 입은 남자가 그릴에서 저녁 식사를 고치고 있다.
빨간 셔츠와 카키스를 입은 남자가 밖에서 저녁을 만들고 있다.


In [ ]:
encoding_result = tokenizer.encode_plus(prem, hypo, max_length=max_seq_len, pad_to_max_length=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
print(encoding_result['input_ids'])

[2, 8013, 10727, 2170, 1697, 2089, 2485, 8802, 2138, 1511, 2073, 3997, 2116, 12222, 27135, 4750, 5067, 2138, 9995, 2088, 1513, 2062, 18, 3, 8013, 10727, 2522, 1697, 16052, 2138, 1511, 2073, 3997, 2116, 4044, 2112, 4750, 2069, 3679, 2088, 1513, 2062, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
print(encoding_result['token_type_ids'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
def convert_examples_to_features(prem_list, hypo_list, max_seq_len, tokenizer):

    input_ids, attention_masks, token_type_ids = [], [], []

    for prem, hypo in tqdm(zip(prem_list, hypo_list), total=len(prem_list)):
        encoding_result = tokenizer.encode_plus(prem, hypo, max_length=max_seq_len, pad_to_max_length=True)

        input_ids.append(encoding_result['input_ids'])
        attention_masks.append(encoding_result['attention_mask'])
        token_type_ids.append(encoding_result['token_type_ids'])

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    return (input_ids, attention_masks, token_type_ids)

In [ ]:
X_train = convert_examples_to_features(train_data['premise'], train_data['hypothesis'], max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|          | 0/797848 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
  1%|▏         | 10236/797848 [00:05<07:14, 1813.98it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So t

In [ ]:
# 최대 길이: 128
input_id = X_train[0][0]
attention_mask = X_train[1][0]
token_type_id = X_train[2][0]

print('단어에 대한 정수 인코딩 :',input_id)
print('어텐션 마스크 :',attention_mask)
print('세그먼트 인코딩 :',token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    2  8013 10727  2170  1697  2089  2485  8802  2138  1511  2073  3997
  2116 12222 27135  4750  5067  2138  9995  2088  1513  2062    18     3
  8013 10727  2522  1697 16052  2138  1511  2073  3997  2116  4044  2112
  4750  2069  3679  2088  1513  2062    18     3     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
X_val = convert_examples_to_features(val_data['premise'], val_data['hypothesis'], max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|          | 0/179462 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
  1%|          | 2185/179462 [00:01<01:40, 1761.93it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So th

In [ ]:
input_id = X_val[0][0]
attention_mask = X_val[1][0]
token_type_id = X_val[2][0]

print('단어에 대한 정수 인코딩 :',input_id)
print('어텐션 마스크 :',attention_mask)
print('세그먼트 인코딩 :',token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    2  4255  2645  2116 14548  2079  4610  7264  2145  4202  2470  3641
  6203  2170  3618  5404 31221  7350 22640  2069 10445  2470   575  2069
  6427  2470  3665  7839  5111  2138  1889  2118  1380  2886  5209    16
  3994  2073  3994  2079   850  2069  1410  2069  1295  1513  2259   831
  1560  2073  4118  2138  5989 20660  5322    18     3  4255  2645  2259
   732  2116  4997  2578  2170  3643  1041  2470   575  2031  2069  3954
  2470  4420  2138  1329  2359  2062    18     3     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
X_test = convert_examples_to_features(test_data['premise'], test_data['hypothesis'], max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|          | 0/1666 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 1666/1666 [00:01<00:00, 1651.33it/s]


In [ ]:
train_label = train_data['label'].tolist()
val_label = val_data['label'].tolist()
test_label = test_data['label'].tolist()

In [ ]:
idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)

y_train = idx_encode.transform(train_label) # 주어진 고유한 정수로 변환
y_val = idx_encode.transform(val_label) # 고유한 정수로 변환
# y_test = idx_encode.transform(test_label) # 고유한 정수로 변환

label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
idx_label = {value: key for key, value in label_idx.items()}
print(label_idx)
print(idx_label)

{'contradiction': 0, 'entailment': 1, 'neutral': 2}
{0: 'contradiction', 1: 'entailment', 2: 'neutral'}


In [ ]:
class TFBertForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFBertForSequenceClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_token = outputs[1]
        prediction = self.classifier(cls_token)

        return prediction

In [ ]:
# TPU 작동을 위한 코드 TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.40.197.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.40.197.130:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
with strategy.scope():
  model = TFBertForSequenceClassification("klue/bert-base", num_labels=3)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
early_stopping = EarlyStopping(
    monitor="val_accuracy", 
    min_delta=0.001,
    patience=2)

model.fit(
    X_train, y_train, epochs=5, batch_size=512, validation_data = (X_val, y_val),
    callbacks = [early_stopping]
)

Epoch 1/10
1559/1559 [==============================] - 751s 423ms/step - loss: 0.5346 - accuracy: 0.7839 - val_loss: 0.4557 - val_accuracy: 0.8242
Epoch 2/10
1559/1559 [==============================] - 582s 373ms/step - loss: 0.4014 - accuracy: 0.8467 - val_loss: 0.4366 - val_accuracy: 0.8321
Epoch 3/10
1559/1559 [==============================] - 582s 374ms/step - loss: 0.3233 - accuracy: 0.8794 - val_loss: 0.4771 - val_accuracy: 0.8316
Epoch 4/10
1559/1559 [==============================] - 582s 373ms/step - loss: 0.2557 - accuracy: 0.9069 - val_loss: 0.5113 - val_accuracy: 0.8291


In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tf_bert_model (TFBertModel)  multiple                 110617344 
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
Total params: 110,619,651
Trainable params: 110,619,651
Non-trainable params: 0
_________________________________________________________________


In [ ]:
pred=model.predict(X_test)
len(pred)

1666

In [ ]:
result=[np.argmax(val) for val in pred]
len(result)
result[0:10]

[0, 2, 1, 0, 0, 2, 2, 1, 1, 2]

In [ ]:
out = [list(label_idx.keys())[_] for _ in result]
out[:3]

['contradiction', 'neutral', 'entailment']

In [ ]:
submission_data['label']=out
submission_data.to_csv("result_TPU.csv", index=False)